In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [2]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))
 
	# blob will be a 4D NumPy array with shape (1, 3, 224, 224), 1 → batch size (just one image),3 → channels (B, G, R),224 → height,224 → width

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()
	print(detections.shape)
 
    #Each detection row has 7 values:[image_id, class_id, confidence, x_min, y_min, x_max, y_max]
    #[[[
    #[0.0, 1.0, 0.956, 0.12, 0.25, 0.40, 0.55],
    #[0.0, 1.0, 0.732, 0.15, 0.28, 0.42, 0.58],
    #[0.0, 1.0, 0.421, 0.18, 0.30, 0.44, 0.60]
    #]]]
    
    
	#0.0 → image ID (since we process one image at a time, always 0)
	#1.0 → class ID (here 1 = face, because model is trained only for faces)
	#0.956 → confidence (probability of being a face, 95.6%)
 
	#detections.shape
 	#4D NumPy array with the shape (1, 1, N, 7) — where N is the number of detected objects.
	#Level 1 → 1 element (everything inside)
	#Level 2 → 1 element (everything inside)
	#Level 3 → 3 elements (3 rows)
	#Level 4 → 7 elements (each row has 7 numbers)
	#✅ Shape = (1, 1, 3, 7)
 
	#1 → batch size (you processed 1 frame)
	#1 → number of classes (here only “face”)
	#N → number of detected objects (could be up to 200, even if many are low confidence)
	#7 → 7 values for each detection ([image_id, class_id, confidence, x_min, y_min, x_max, y_max])
 
	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
   			#Ensures the top-left corner is not outside the image.If startX or startY is negative (which can happen due to model prediction noise), it replaces it with 0
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))
			#If endX or endY is larger than the image width/height, it clamps them to the maximum valid pixel index.
   
			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			#OpenCV reads images in BGR format by default.
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			# add the face and bounding boxes to their respective
			# lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# only make a predictions if at least one face was detected
	if len(faces) > 0:
		# for faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	# return a 2-tuple of the face locations and their corresponding
	# locations
	return (locs, preds)

In [3]:
# load our serialized face detector model from disk
prototxtPath =r"C:\Users\jeeva\Documents\Machine Learning\Projects\Projects\Video Based Facemask Detection\face_detector\deploy.prototxt"
weightsPath = r"C:\Users\jeeva\Documents\Machine Learning\Projects\Projects\Video Based Facemask Detection\face_detector\res10_300x300_ssd_iter_140000.caffemodel"
#prototxt = structure of the neural network
#weights file / .caffemodel = Recognize faces and Make predictions 
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [4]:
# load the face mask detector model from disk
maskNet = load_model("mask_detector.keras")

In [5]:
# initialize the video stream
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()

# loop over the frames from the video stream
while True:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 400 pixels
	frame = vs.read()
	frame = imutils.resize(frame, width=400)

	# detect faces in the frame and determine if they are wearing a
	# face mask or not
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	# loop over the detected face locations and their corresponding
	# locations
	for (box, pred) in zip(locs, preds):
		# unpack the bounding box and predictions
		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred

		# determine the class label and color we'll use to draw
		# the bounding box and text
		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		# include the probability in the label
		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		# display the label and bounding box rectangle on the output
		# frame
		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	# show the output frame
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	# if the `q` key was pressed, break from the loop
	if key == ord("q"): 
		break
	# ord("q") → converts the character 'q' to its ASCII code (which is 113).


# do a bit of cleanup
cv2.destroyAllWindows()#Closes all windows opened by OpenCV (like the live video feed).
vs.stop()

[INFO] starting video stream...
(1, 1, 200, 7)
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
(